In [38]:
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
import time

In [39]:
data = load_breast_cancer()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [40]:
grad = GradientBoostingClassifier(random_state=42)
ada = AdaBoostClassifier(random_state=42)
xgb = XGBClassifier(eval_metric='logloss', random_state=42)


results = {}

start = time.time()
grad.fit(X_train, y_train)
end = time.time()

y_pred = grad.predict(X_test)
y_proba = grad.predict_proba(X_test)[:, 1]

results['Gradient'] = {
    'Accuracy': accuracy_score(y_test, y_pred),
    'ROC-AUC': roc_auc_score(y_test, y_proba),
    'Training Time (s)': end - start
}

start = time.time()
ada.fit(X_train, y_train)
end = time.time()

y_pred = ada.predict(X_test)
y_proba = ada.predict_proba(X_test)[:, 1]

results['ADA'] = {
    'Accuracy': accuracy_score(y_test, y_pred),
    'ROC-AUC': roc_auc_score(y_test, y_proba),
    'Training Time (s)': end - start
}

start = time.time()
xgb.fit(X_train, y_train)
end = time.time()

y_pred = xgb.predict(X_test)
y_proba = xgb.predict_proba(X_test)[:, 1]

results['XGB'] = {
    'Accuracy': accuracy_score(y_test, y_pred),
    'ROC-AUC': roc_auc_score(y_test, y_proba),
    'Training Time (s)': end - start
}

pd.DataFrame(results).T

,Accuracy,ROC-AUC,Training Time (s)
Gradient,0.956140,0.990741,0.703904
ADA,0.956140,0.982474,0.214629
XGB,0.947368,0.991733,0.089836


In [42]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Load and split data
data = load_breast_cancer()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Further split training data to create a validation set
X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

# Define the model
model = xgb.XGBClassifier(
    n_estimators=1000,
    learning_rate=0.05,
    eval_metric='logloss',
    use_label_encoder=False
)

# Fit the model with early stopping
model.fit(
    X_train_final, y_train_final,
    eval_set=[(X_val, y_val)],
    early_stopping_rounds=10,
    verbose=True
)

# Evaluate on the test set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nTest Accuracy: {accuracy:.4f}")


TypeError: XGBClassifier.fit() got an unexpected keyword argument 'early_stopping_rounds'